In [127]:
import pandas as pd
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard 
from datetime import datetime
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder 
import pickle  
label_encoder_gender = LabelEncoder()
one_hot_encoder = OneHotEncoder(sparse_output = False)

In [128]:
churn = pd.read_csv('Churn_modelling.csv')
churn = churn.drop(columns = ['RowNumber', 'CustomerId', 'Surname'])
churn.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [129]:
cols = churn.select_dtypes(include = 'object').columns

churn[cols] = churn[cols].astype('category')



In [130]:
idsx = churn.select_dtypes(include = 'category').columns
for i in idsx:
    if len(churn[i].cat.categories) <= 2:
        churn[i] =  label_encoder_gender.fit_transform(churn[i])
    else:
        churn[i] = churn[i]


In [131]:
geo_df = one_hot_encoder.fit_transform(churn[['Geography']])

geo_df = pd.DataFrame(geo_df, columns = one_hot_encoder.get_feature_names_out(['Geography']))
churn = pd.concat([churn.drop(columns = 'Geography'), geo_df], axis = 1)

In [132]:
churn

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [133]:
with open('label_encoder_gender.pk1', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_hot_encoder.pk1', 'wb') as file:
    pickle.dump(one_hot_encoder, file)

In [134]:
y = churn['Exited']
x = churn.drop(columns = 'Exited')


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, )


scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [135]:
with open('scaler.pk1', 'wb') as file:
    pickle.dump(scaler, file)

In [136]:
model = Sequential([
    Dense(64, activation = 'relu', input_shape = (x_train.shape[1],)),
    Dense(32, activation = 'relu'),
    Dense(1, activation = 'sigmoid')         
])

In [137]:
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [138]:
model.compile(optimizer = opt, loss = loss, metrics = ['accuracy'])

In [139]:
log_dir = 'logs/fit/' + datetime.now().strftime("%m_%d_%Y_%S_%M_%H_")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)

In [140]:
history = model.fit(
    x_train, y_train, validation_data = (x_test, y_test), epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3954 - accuracy: 0.8334 - val_loss: 0.3385 - val_accuracy: 0.8655
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3542 - accuracy: 0.8549 - val_loss: 0.3343 - val_accuracy: 0.8680
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3444 - accuracy: 0.8585 - val_loss: 0.3487 - val_accuracy: 0.8590
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3440 - accuracy: 0.8601 - val_loss: 0.3458 - val_accuracy: 0.8580
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3404 - accuracy: 0.8581 - val_loss: 0.3352 - val_accuracy: 0.8625
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3356 - accuracy: 0.8646 - val_loss: 0.3405 - val_accuracy: 0.8610
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3359 - accuracy: 0.8616 - val_loss: 0.3362 - val_accuracy: 0.8590

In [141]:
model.save('model.h5')

c:\Users\tariq\Desktop\Neural Network Project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [142]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [143]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 44596), started 0:49:33 ago. (Use '!kill 44596' to kill it.)